In [12]:
# Приближённое вычисление интеграла по составным квадратурным формулам

In [13]:
rho(x) = 1
f(x) = rho(x) * log(1 + x)
function df(x, order)
    if order == 1
        return 1 / (1 + x)
    elseif order == 2
        return -1 / ((1 + x) ^ 2)
    elseif order == 3    
        return 2 / ((1 + x) ^ 3)
    elseif order == 4
        return -6 / ((1 + x) ^ 4) 
    else
        throw(e)
    end
end

intf(x) = (1 + x) * log(1 + x) - x
pure_int(a, b) = intf(b) - intf(a)

(a, b, m) = (50, 100, 1e4)
h = (b - a) / m
r = a:h:b
function q_left(fn)
    sum = 0
    for i = 0:m-1
        sum += fn(a + i * h) * h
    end
    return sum
end

function q_right(fn)
    sum = 0
    for i = 1:m
        sum += fn(a + i * h) * h
    end
    return sum
end

function q_center(fn)
    sum = 0
    for i = 0:m-1
        sum += f(a + (i + 1/2) * h) * h
    end
    return sum
end

function q_trapez(fn)
    sum = 0
    sum += f(a) + f(b)
    for i = 1:m-1
        sum += 2 * fn(a + i * h)
    end
    sum *= h / 2
    return sum
end

function q_simpson(fn)
    sum = 0
    for i = 1:2:m-1
        sum += h / 3 * (fn(a+(i-1) * h) + 4 * fn(a + i * h) + fn(a + (i+1)*h))
        end
    return sum
end

q_simpson (generic function with 1 method)

In [14]:
header = (["exact_int", "q_left", "q_right", "q_center", "q_trapez", "q_simpson"])
using PrettyTables
new_f = (x) -> f(x)
exact = pure_int(a, b)
res = [q_left(new_f), q_right(new_f), q_center(new_f), q_trapez(new_f), q_simpson(new_f)]
data = hcat(exact, res...)
results_table = pretty_table(data, header = header)

┌───────────┬─────────┬─────────┬──────────┬──────────┬───────────┐
│ exact_int │  q_left │ q_right │ q_center │ q_trapez │ q_simpson │
├───────────┼─────────┼─────────┼──────────┼──────────┼───────────┤
│   215.604 │ 215.602 │ 215.606 │  215.604 │  215.604 │   215.604 │
└───────────┴─────────┴─────────┴──────────┴──────────┴───────────┘


In [15]:
header = (["exact_int", "q_left_err", "q_right_err", "q_center_err", "q_trapez_err", "q_simpson_err"])
err = hcat(exact, map(abs, res .- exact)...)
err_table = pretty_table(err, header=header)
println(exact - res[5])

┌───────────┬────────────┬─────────────┬──────────────┬──────────────┬───────────────┐
│ exact_int │ q_left_err │ q_right_err │ q_center_err │ q_trapez_err │ q_simpson_err │
├───────────┼────────────┼─────────────┼──────────────┼──────────────┼───────────────┤
│   215.604 │ 0.00170826 │  0.00170822 │   1.01119e-8 │   2.02224e-8 │   2.55795e-13 │
└───────────┴────────────┴─────────────┴──────────────┴──────────────┴───────────────┘
2.5579538487363607e-13


In [16]:
header = (["exact_int", "t_q_left_err", "t_q_right_err", "t_q_center_err", "t_q_trapez_err", "t_q_simpson_err"])
theor_err(c, ast) = c * maximum(map(abs, df.(r, ast + 1))) * (b - a) * (h ^ (ast + 1))
theor_err_list = [theor_err(0.5, 0), theor_err(0.5, 0), theor_err(1/24, 1), theor_err(1/12, 1), theor_err(1/2880, 3)]
err = hcat(exact, theor_err_list...)
err_table = pretty_table(err, header=header)

┌───────────┬──────────────┬───────────────┬────────────────┬────────────────┬─────────────────┐
│ exact_int │ t_q_left_err │ t_q_right_err │ t_q_center_err │ t_q_trapez_err │ t_q_simpson_err │
├───────────┼──────────────┼───────────────┼────────────────┼────────────────┼─────────────────┤
│   215.604 │   0.00245098 │    0.00245098 │     2.00243e-8 │     4.00487e-8 │     9.62339e-18 │
└───────────┴──────────────┴───────────────┴────────────────┴────────────────┴─────────────────┘
